In [5]:
import os
import pandas as pd
import sys

In [6]:
sys.path.append(os.path.abspath('../..'))

In [7]:
from analysis.preprocessing import load_and_clean_red_list_index_data

In [8]:
df = load_and_clean_red_list_index_data()
df

,Entity,Code,Year,_15_5_1__er_rsk_lst
0,Afghanistan,AFG,1993,0.93
1,Afghanistan,AFG,1994,0.93
2,Afghanistan,AFG,1995,0.93
3,Afghanistan,AFG,1996,0.93
4,Afghanistan,AFG,1997,0.93
...,...,...,...,...
8315,Zimbabwe,ZWE,2020,0.79
8316,Zimbabwe,ZWE,2021,0.79
8317,Zimbabwe,ZWE,2022,0.79
8318,Zimbabwe,ZWE,2023,0.79
